In [54]:
import os
import pickle
import torch
import exp_configs

In [37]:
import wandb

entity = 'cnn'
project = 'multilabel_ar_hmdb_202207'
api = wandb.Api()

runs = api.runs(f"{entity}/{project}")

In [40]:
# find the run
search_run_names = ['hmdb_confusion2 tsm_resnet50_nopartialbn entropy_maximise split1', 'hmdb_confusion2 ch_epic100.tsm_resnet50_flow entropy_maximise split1']
found_runs = []
for run in runs:
    for search_run_name in search_run_names:
        if search_run_name in run.name:
            print(run.name, run.id)
            found_runs.append(run)

assert len(found_runs) == len(search_run_names)

predictions = []
for run in found_runs:
    # find artifact from the run.
    produced_artifacts = run.logged_artifacts()
    for art in produced_artifacts:
        if art.name.startswith('predictions'):
            print(art.name)
            artifact_name = art.name

    # Download the artifact
    artifact = api.artifact(f"{entity}/{project}/{artifact_name}")
    artifact_dir = artifact.checkout()

    # open
    best_epoch = run.summary['epoch_best-val_epoch']
    with open(os.path.join(artifact_dir, f'epoch_{best_epoch:04d}_val_oneclip.pkl'), 'rb') as f:
        data = pickle.load(f)
    predictions.append(data)

hmdb_confusion2 ch_epic100.tsm_resnet50_flow entropy_maximise split1 v0 3aapmfba
hmdb_confusion2 tsm_resnet50_nopartialbn entropy_maximise split1 v0 2ec4kzj7
predictions:v4
predictions:v0


In [50]:
# Compute sigmoid average of two predictions
assert all(predictions[0]['video_ids'] == predictions[1]['video_ids'])
sigmoid_1 = torch.sigmoid(torch.tensor(predictions[0]['video_predictions']))
sigmoid_2 = torch.sigmoid(torch.tensor(predictions[1]['video_predictions']))
sigmoid_average = (sigmoid_1 + sigmoid_2) / 2

In [52]:
sigmoid_1

tensor([[0.8446, 0.5062, 0.5175,  ..., 0.4825, 0.4943, 0.5390],
        [0.8317, 0.4956, 0.4772,  ..., 0.4597, 0.4909, 0.4940],
        [0.8701, 0.4786, 0.4624,  ..., 0.4663, 0.5124, 0.5425],
        ...,
        [0.5350, 0.4910, 0.4853,  ..., 0.5337, 0.5271, 0.5346],
        [0.5356, 0.4997, 0.5058,  ..., 0.5085, 0.5292, 0.5110],
        [0.4181, 0.5011, 0.4826,  ..., 0.5248, 0.4769, 0.6315]])

In [53]:
sigmoid_2

tensor([[0.9356, 0.5135, 0.5104,  ..., 0.4688, 0.5011, 0.5616],
        [0.8533, 0.5421, 0.4915,  ..., 0.5325, 0.5755, 0.5171],
        [0.8798, 0.5573, 0.4774,  ..., 0.4899, 0.4993, 0.6170],
        ...,
        [0.5271, 0.5809, 0.4670,  ..., 0.5971, 0.6448, 0.5476],
        [0.5294, 0.5826, 0.4852,  ..., 0.5951, 0.6115, 0.5807],
        [0.5578, 0.5020, 0.4951,  ..., 0.6422, 0.5599, 0.6163]])

In [51]:
sigmoid_average

tensor([[0.8901, 0.5099, 0.5140,  ..., 0.4756, 0.4977, 0.5503],
        [0.8425, 0.5189, 0.4844,  ..., 0.4961, 0.5332, 0.5055],
        [0.8750, 0.5180, 0.4699,  ..., 0.4781, 0.5058, 0.5797],
        ...,
        [0.5310, 0.5360, 0.4761,  ..., 0.5654, 0.5860, 0.5411],
        [0.5325, 0.5411, 0.4955,  ..., 0.5518, 0.5703, 0.5458],
        [0.4880, 0.5015, 0.4889,  ..., 0.5835, 0.5184, 0.6239]])

In [68]:
# calculate metrics
cfg = exp_configs.load_cfg('hmdb_confusion2', 'tsm_resnet50_nopartialbn', 'assume_negative', dataset_channel='beta', exp_channel='labelsmth')

for metric in cfg.metrics['val']:
    metric.clean_data()
    metric.add_clip_predictions(torch.tensor(predictions[0]['video_ids']), sigmoid_average, torch.tensor(predictions[0]['video_labels']))
    metric.calculate_metrics()
    print(metric.last_calculated_metrics)

0.611764705882353
0.6294117647058823
0.6836601307189543
0.637563587725868
